In [1]:
from nltk import wsd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import warnings

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')

[nltk_data] Downloading package omw-1.4 to /home/dante/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dante/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     /home/dante/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!


True

In [2]:
nlp = load('en_core_web_sm')

In [3]:
"""En el siguiente ejemplo la palabra morir tiene un significado diferente en cada frase.
Sólo entendiendo el contexto de la palabra la PNL puede improvisar más.
"""

X = 'The die is cast.' # La suerte está echada. // El dado está lanzado ?? 
Y = 'Roll the die to get a 6.' # Tira el dado para obtener un 6.
Z = 'What is dead may never die.' # Lo que está muerto nunca morirá.

""" En este ejemplo se  va utilizar wordnet de la universidad de princeton para obtener de la palabra diferentes frases de contexto como parte de la oración adjunta.
 ->> Wordnet : es una base de datos léxica del inglés. Sustantivos, verbos, adjetivos y adverbios que se agrupan en conjuntos de  sinónimos cognitivos (synsets), cada uno de los cuales expresa un concepto distinto.
 ->> En python los datos de wordnet se cargan con NLTK.
 ->> Por ejemplo se pasa la palabra die a wordnet y se intenta obtener las diferentes frases unqiue wordnet que se tiene para die
 ->> La salida de wordnet puede ser definiciones diferentes de die que incluyen verbos y sustantivos
"""

wn.synsets("die")

# comprobar detalles relacionados con el sustantivo
wn.synsets('die', pos=wn.NOUN)

# imprimir todas las definiciones relacionados con  sustantivos
i =0
for syn in wn.synsets('die', pos=wn.NOUN):
    print("definicion {0} : {1}".format(i, syn.definition()))
    i+=1

definicion 0 : a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers
definicion 1 : a device used for shaping metal
definicion 2 : a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods


In [4]:
# imprimir todas las definiciones relacionados con verbos
i =0
for syn in wn.synsets('die', pos=wn.VERB):
    print("defination {0} : {1}".format(i, syn.definition()))
    i+=1

defination 0 : pass from physical life and lose all bodily attributes and functions necessary to sustain life
defination 1 : suffer or face the pain of death
defination 2 : be brought to or as if to the point of death by an intense emotion such as embarrassment, amusement, or shame
defination 3 : stop operating or functioning
defination 4 : feel indifferent towards
defination 5 : languish as with love or desire
defination 6 : cut or shape with a die
defination 7 : to be on base at the end of an inning, of a player
defination 8 : lose sparkle or bouquet
defination 9 : disappear or come to an end
defination 10 : suffer spiritual death; be damned (in the religious sense)


In [31]:
"""Word-Sense Disambiguation con Algoritmo Lesk"""

# introduzca la frase X "La suerte está echada" y compruebe si lesk es capaz de encontrar la frase similar correcta.
print(X)
print(wsd.lesk(X.split(), 'die'))
print(wsd.lesk(X.split(), 'die').definition())

"""Para la frase de entrada X,lesk ha encontrado una frase similar coincidente cuyo tipo es verbo, y que no es correcta. En la siguiente se pasará explícitamente la parte de la oración (POS) y se comprobará la salida 

La POS tagging (Part of Speech" en inglés) es un proceso en el que cada palabra en un texto es etiquetada con su categoría gramatical, como sustantivo, verbo, adjetivo, etc.
"""


The die is cast.
Synset('die.v.07')
cut or shape with a die


'Para la frase de entrada X,lesk ha encontrado una frase similar coincidente cuyo tipo es verbo, y que no es correcta. En la siguiente se pasará explícitamente la parte de la oración (POS) y se comprobará la salida \n\nLa POS tagging (Part of Speech" en inglés) es un proceso en el que cada palabra en un texto es etiquetada con su categoría gramatical, como sustantivo, verbo, adjetivo, etc.\n'

In [6]:
# al pasar un POS conseguimos la definición correcta
print(X)
wsd.lesk(X.split(), 'die', pos=wn.NOUN).definition()

The die is cast.


'a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods'

In [7]:
# para la frase X, es decir, "Tira el dado para obtener un 6." es de nuevo un sustantivo.
print(Y)
wsd.lesk(Y.split(), 'die').definition()

Roll the die to get a 6.


'to be on base at the end of an inning, of a player'

In [8]:
# Con pasar un POS conseguimos la defianción correcta?
wsd.lesk(Y.split(), 'die', pos=wn.NOUN).definition()

'a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers'

In [28]:
# Observaciones similares con la sentencia Z
print(Z)
print(wsd.lesk(Z.split(), 'die').definition(),"\n")
print(wsd.lesk(Z.split(), 'die', pos=wn.VERB).definition())

What is dead may never die.
a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods 

stop operating or functioning


In [16]:
"""Etiquetado automático"""

# Pasar una frase a spacy para la búsqueda automática de la etiqueta POS.
# Observar la salida, es evidente que la forma POS se encuentra para una frase.

sent1 = "I loved the screen on this phone."
doc1 = nlp(sent1)
for tok in doc1:
    print(tok.text,tok.pos_)

I PRON
loved VERB
the DET
screen NOUN
on ADP
this DET
phone NOUN
. PUNCT


In [17]:
# ejemplos adicionales
sent2 = "The battery life on this phone is great. !"
doc2 = nlp(sent2)
for tok in doc2:
    print(tok.text,tok.pos_)

The DET
battery NOUN
life NOUN
on ADP
this DET
phone NOUN
is AUX
great ADJ
. PUNCT
! PUNCT


In [18]:
# leer el etiquetado POS y el lema (sólo ampliando un ejemplo para mostrar el lema)

sent1 = "I loved the screen on this phone."
doc1 = nlp(sent1)
pos = []
lemma = []
text = []
for tok in doc1:
    pos.append(tok.pos_)
    lemma.append(tok.lemma_)
    text.append(tok.text)
nlp_table = pd.DataFrame({'text':text,'lemma':lemma,'pos':pos})
nlp_table.head()

,text,lemma,pos
0,I,I,PRON
1,loved,love,VERB
2,the,the,DET
3,screen,screen,NOUN
4,on,on,ADP


In [21]:
"""Etiquetado Automatico POS + Lesk con spaCy"""

POS_MAP = {
    'VERB': wn.VERB,
    'NOUN': wn.NOUN,
    'PROPN': wn.NOUN
}

def lesk(doc, word):
    found = False
    for token in doc:
        if token.text == word:
            word = token
            found = True
            break
    if not found:
        raise ValueError(f'Word \"{word}\" No aparece en el documentos: {doc.text}.')
    pos = POS_MAP.get(word.pos_, False)
    if not pos:
        warnings.warn(f'POS tag para {word.text} no encontrada en wordnet. Volver al comportamiento por defecto de Lesk.')
    args = [c.text for c in doc], word.text
    kwargs = dict(pos=pos)
    return wsd.lesk(*args, **kwargs)

doc = nlp('Roll the die to get a 6.')
lesk(doc, 'die')

Synset('die.n.01')

In [22]:
# búsqueda de la etiqueta POS por defecto, para ayudar a lesk a encontrar la definición correcta.

lesk(doc, 'die').definition()

'a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers'

In [24]:
# ejemplo adicional, con la sentencia siguiente

lesk(nlp('I work at google.'), 'google').definition()

"a widely used search engine that uses text-matching techniques to find web pages that are important and relevant to a user's search"

In [25]:
# revisar cuando se trata como verbo google

lesk(nlp('I will google it.'), 'google').definition()

'search the internet (for information) using the Google search engine'

In [26]:
# esperanza como sustantivo
lesk(nlp('Her pep talk gave me hope'), 'hope').definition()

'grounds for feeling hopeful about the future'

In [27]:
# esperanza como verbo
lesk(nlp('I hope we win!'), 'hope').definition()

'intend with some possibility of fulfilment'